In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [2]:
cur_dir= 'C:/Users/Hp/Personal_coding/mask_detection/'

In [3]:
prototxtPath=os.path.sep.join([cur_dir + r'deploy.prototxt'])
weightsPath=os.path.sep.join([cur_dir + r'res10_300x300_ssd_iter_140000.caffemodel'])

In [4]:
prototxtPath

'C:/Users/Hp/Personal_coding/mask_detection/deploy.prototxt'

In [5]:
weightsPath

'C:/Users/Hp/Personal_coding/mask_detection/res10_300x300_ssd_iter_140000.caffemodel'

In [6]:
net=cv2.dnn.readNet(prototxtPath,weightsPath)

## Loading the trained model

In [7]:
model=load_model(cur_dir + r'custom_trained_mobilenet_v2.model')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
image=cv2.imread(cur_dir+ r'example_02.png')

In [9]:
image

array([[[235, 235, 235],
        [235, 235, 235],
        [235, 235, 235],
        ...,
        [169, 169, 169],
        [169, 169, 169],
        [169, 169, 169]],

       [[235, 235, 235],
        [235, 235, 235],
        [235, 235, 235],
        ...,
        [169, 169, 169],
        [169, 169, 169],
        [169, 169, 169]],

       [[235, 235, 235],
        [235, 235, 235],
        [235, 235, 235],
        ...,
        [169, 169, 169],
        [169, 169, 169],
        [169, 169, 169]],

       ...,

       [[169, 169, 169],
        [169, 169, 169],
        [170, 170, 170],
        ...,
        [118, 118, 118],
        [117, 117, 117],
        [116, 116, 116]],

       [[169, 169, 169],
        [169, 169, 169],
        [170, 170, 170],
        ...,
        [116, 116, 116],
        [115, 115, 115],
        [114, 114, 114]],

       [[169, 169, 169],
        [170, 170, 170],
        [170, 170, 170],
        ...,
        [115, 115, 115],
        [113, 113, 113],
        [112, 112, 112]]

In [10]:
(h,w)=image.shape[:2]

In [11]:
(h,w)

(400, 400)

In [12]:
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

In [13]:
blob

array([[[[131., 131., 131., ...,  66.,  65.,  65.],
         [131., 131., 131., ...,  66.,  65.,  65.],
         [131., 131., 131., ...,  66.,  65.,  65.],
         ...,
         [ 65.,  65.,  67., ...,  16.,  14.,  14.],
         [ 65.,  66.,  67., ...,  14.,  13.,  11.],
         [ 65.,  66.,  67., ...,  13.,  10.,   8.]],

        [[ 58.,  58.,  58., ...,  -7.,  -8.,  -8.],
         [ 58.,  58.,  58., ...,  -7.,  -8.,  -8.],
         [ 58.,  58.,  58., ...,  -7.,  -8.,  -8.],
         ...,
         [ -8.,  -8.,  -6., ..., -57., -59., -59.],
         [ -8.,  -7.,  -6., ..., -59., -60., -62.],
         [ -8.,  -7.,  -6., ..., -60., -63., -65.]],

        [[112., 112., 112., ...,  47.,  46.,  46.],
         [112., 112., 112., ...,  47.,  46.,  46.],
         [112., 112., 112., ...,  47.,  46.,  46.],
         ...,
         [ 46.,  46.,  48., ...,  -3.,  -5.,  -5.],
         [ 46.,  47.,  48., ...,  -5.,  -6.,  -8.],
         [ 46.,  47.,  48., ...,  -6.,  -9., -11.]]]], dtype=float32)

In [14]:
blob.shape

(1, 3, 300, 300)

In [15]:
net.setInput(blob)
detections=net.forward()

In [16]:
detections

array([[[[0.        , 1.        , 0.99923694, ..., 0.12586218,
          0.6852968 , 0.43432277],
         [0.        , 1.        , 0.12769319, ..., 4.003238  ,
          4.8399506 , 4.985212  ],
         [0.        , 1.        , 0.12365586, ..., 4.0013595 ,
          0.83174324, 4.9895177 ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

## Predicting the output using CV2

In [17]:
#loop over the detections
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    
    if confidence>0.5:
        #we need the X,Y coordinates
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        #ensure the bounding boxes fall within the dimensions of the frame
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
        
        #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
        face=image[startY:endY, startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        #determine the class label and color we will use to draw the bounding box and text
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        #include the probability in the label
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        #display the label and bounding boxes
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
        
        
cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyAllWindows()